In [16]:
%%capture
!mamba install --force-reinstall aiohttp -y
!pip install -U "xformers<0.0.26" --index-url https://download.pytorch.org/whl/cu121
!pip install "unsloth[kaggle-new] @ git+https://github.com/unslothai/unsloth.git"

# Temporary fix for https://github.com/huggingface/datasets/issues/6753
!pip install datasets==2.16.0 fsspec==2023.10.0 gcsfs==2023.10.0

In [4]:
from kaggle_secrets import UserSecretsClient

In [5]:
user_secrets = UserSecretsClient()
token = user_secrets.get_secret("GITHUB_KEY")

In [6]:
!git clone https://{token}@github.com/shreeya-dhakal/llama-3-finetune.git
    

Cloning into 'llama-3-finetune'...
remote: Enumerating objects: 54, done.
remote: Counting objects: 100% (54/54), done.
remote: Compressing objects: 100% (40/40), done.
remote: Total 54 (delta 13), reused 44 (delta 6), pack-reused 0
Unpacking objects: 100% (54/54), 27.01 MiB | 6.54 MiB/s, done.


In [7]:
%cd /kaggle/working/llama-3-finetune

/kaggle/working/llama-3-finetune


In [8]:
!git checkout -b Update-Eval-Notebook

Switched to a new branch 'Update-Eval-Notebook'


In [10]:
!git pull origin Update-Eval-Notebook

From https://github.com/shreeya-dhakal/llama-3-finetune
 * branch            Update-Eval-Notebook -> FETCH_HEAD
Updating 0b9d6ce..4a61355
Fast-forward
 data_splits/alpaca-cleaned-hindi_test_split.txt  | 10352 +++++
 data_splits/alpaca-cleaned-hindi_train_split.txt | 41408 ++++++++++++++++++++
 data_splits/alpaca-nepali-sft_test_split.txt     | 10392 ++++++
 data_splits/alpaca-nepali-sft_train_split.txt    | 41567 +++++++++++++++++++++
 data_splits/aya_dataset_hindi_test_split.txt     |   231 +
 data_splits/aya_dataset_hindi_train_split.txt    |   922 +
 data_splits/aya_dataset_nepali_test_split.txt    |   801 +
 data_splits/aya_dataset_nepali_train_split.txt   |  3201 ++
 src/__init__.py                                  |     0
 src/data/__init__.py                             |     0
 src/data/data_sampler.py                         |    75 +
 src/notebooks/llama3_eval.ipynb                  |    24 -
 src/notebooks/llama3_finetune.ipynb              |    22 +
 src/notebooks/llama3_in

In [21]:
from datasets import load_dataset
import sys
import os
sys.path.append('/kaggle/working/llama-3-finetune/src/')
from data.data_sampler import sample_data
from unsloth import FastLanguageModel
import torch

In [22]:
split_ratio = 0.2
seed = 42

In [23]:
dataset_name = "CohereForAI/aya_dataset"
dataset_type = "aya"

In [24]:
lang_multi = "Nepali"
_, aya_nepali_test = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)

Generating train split:   0%|          | 0/202362 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1750 [00:00<?, ? examples/s]

Filter:   0%|          | 0/202362 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1750 [00:00<?, ? examples/s]

Filter:   0%|          | 0/4002 [00:00<?, ? examples/s]

In [25]:
aya_nepali_test

Dataset({
    features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
    num_rows: 801
})

In [26]:
lang_multi = "Hindi"
_, aya_hindi_test = sample_data(dataset_name, dataset_type, split_ratio, seed, output_dir=None, lang_multi=lang_multi)

Filter:   0%|          | 0/202362 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1750 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1153 [00:00<?, ? examples/s]

In [27]:
aya_hindi_test

Dataset({
    features: ['inputs', 'targets', 'language', 'language_code', 'annotation_type', 'user_id'],
    num_rows: 231
})

In [28]:
dataset_name = "Saugatkafley/alpaca-nepali-sft"
_, alpaca_nepali_test = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)

Generating train split:   0%|          | 0/52005 [00:00<?, ? examples/s]

Filter:   0%|          | 0/52005 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51963 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51963 [00:00<?, ? examples/s]

In [10]:
alpaca_nepali_test

Dataset({
    features: ['instruction', 'input', 'output', 'id'],
    num_rows: 10392
})

In [29]:
dataset_name = "iamshnoo/alpaca-cleaned-hindi"
_, alpaca_hindi_test = sample_data(dataset_name, "alpaca", split_ratio, seed, output_dir=None, lang_multi=None)

Generating train split:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

Filter:   0%|          | 0/51760 [00:00<?, ? examples/s]

In [30]:
alpaca_hindi_test

Dataset({
    features: ['input', 'instruction', 'output'],
    num_rows: 10352
})

In [35]:
alpaca_prompt = """Below is an instruction in hindi that describes a task, paired with an input also in hindi that provides further context. Write a response in hindi that appropriately completes the request.

### Instruction:
{}

### Input:
{}

### Response:
{}"""

In [31]:
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Llama patching release 2024.5
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2+cu121. CUDA = 7.5. CUDA Toolkit = 12.1.
\        /    Bfloat16 = FALSE. Xformers = 0.0.25.post1. FA = False.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/50.6k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/464 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [40]:
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
alpaca_hindi_test_s = alpaca_hindi_test.select(range(10)) 
for i, row in enumerate(alpaca_hindi_test_s):
    instruction = row["instruction"]
    input_text = row["input"]

    prompt = alpaca_prompt.format(instruction, input_text, "")
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    outputs = model.generate(**inputs, max_new_tokens=64, use_cache=True)
    generated_text = tokenizer.batch_decode(outputs)[0]
    alpaca_hindi_test_s[i]["output"] = generated_text

    print(f"Instruction: {instruction}")
    print(f"Input: {input_text}")
    print(f"Generated output: {generated_text}")
    print("---")

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: निम्नलिखित वाक्य को और अधिक दिलचस्प बनाने के लिए पुनः व्यवस्थित करें।
Input: वह पार्टी से जल्दी चली गई
Generated output: <|begin_of_text|>Below is an instruction in hindi that describes a task, paired with an input also in hindi that provides further context. Write a response in hindi that appropriately completes the request.

### Instruction:
निम्नलिखित वाक्य को और अधिक दिलचस्प बनाने के लिए पुनः व्यवस्थित करें।

### Input:
वह पार्टी से जल्दी चली गई

### Response:
वह जल्दी चली गई पार्टी से

## Explanation
In the instruction, the subject "वह" is placed at the end of the sentence, and the verb "चली गई" is placed at the beginning of the sentence. The subject and verb are separated by the pre
---


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: f(x) = {[ -x - 3 यदि x ≤ 1, x/2 + 1 यदि x > 1. ]. x के सभी मानों का योग ज्ञात करें जैसे कि f(x) = 0।
Input: 
Generated output: <|begin_of_text|>Below is an instruction in hindi that describes a task, paired with an input also in hindi that provides further context. Write a response in hindi that appropriately completes the request.

### Instruction:
f(x) = {[ -x - 3 यदि x ≤ 1, x/2 + 1 यदि x > 1. ]. x के सभी मानों का योग ज्ञात करें जैसे कि f(x) = 0।

### Input:


### Response:
यदि x ≤ 1 तो f(x) = -x - 3
यदि x > 1 तो f(x) = x/2 + 1

यदि x ≤ 1 तो x का योग = -x - 3
यदि x
---


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

Instruction: x का मान ज्ञात कीजिए जब 3 ((2x+1) = 15 हो।
Input: 
Generated output: <|begin_of_text|>Below is an instruction in hindi that describes a task, paired with an input also in hindi that provides further context. Write a response in hindi that appropriately completes the request.

### Instruction:
x का मान ज्ञात कीजिए जब 3 ((2x+1) = 15 हो।

### Input:


### Response:
<|end_of_text|>
---


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: इस वाक्य को तीन सुसंगत भागों में विभाजित करें।
Input: हम सब गलतियाँ करते हैं, लेकिन उनसे सीखना महत्वपूर्ण है।
Generated output: <|begin_of_text|>Below is an instruction in hindi that describes a task, paired with an input also in hindi that provides further context. Write a response in hindi that appropriately completes the request.

### Instruction:
इस वाक्य को तीन सुसंगत भागों में विभाजित करें।

### Input:
हम सब गलतियाँ करते हैं, लेकिन उनसे सीखना महत्वपूर्ण है।

### Response:
हम सब गलतियाँ करते हैं, लेकिन उनसे सीखना महत्वपूर्ण है।

### Explanation:
The instruction asks you to divide the sentence into three coherent parts. The input provides the sentence that needs to be divided. The response should divide the sentence into three coherent parts.<|end_of_text|>
---


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Instruction: दक्षिण अमेरिका में स्थित चार देशों का नाम बताइए।
Input: 
Generated output: <|begin_of_text|>Below is an instruction in hindi that describes a task, paired with an input also in hindi that provides further context. Write a response in hindi that appropriately completes the request.

### Instruction:
दक्षिण अमेरिका में स्थित चार देशों का नाम बताइए।

### Input:


### Response:
ब्राज़ील, अर्जेंटीना, उरुग्वे और चिली<|end_of_text|>
---
Instruction: पोषण संबंधी जानकारी के आधार पर खाद्य पदार्थ को वर्गीकृत करें।
Input: कैलोरी: 100। वसाः 2 ग्राम। कार्बोहाइड्रेट: 20 ग्राम। प्रोटीन: 4 ग्राम
Generated output: <|begin_of_text|>Below is an instruction in hindi that describes a task, paired with an input also in hindi that provides further context. Write a response in hindi that appropriately completes the request.

### Instruction:
पोषण संबंधी जानकारी के आधार पर खाद्य पदार्थ को वर्गीकृत करें।

### Input:
कैलोरी: 100। वसाः 2 ग्राम। कार्बोहाइड्रेट: 20 ग्राम। प्रोटीन: 4 ग्राम

### Response:

In [41]:
alpaca_hindi_test_s.to_csv("test_hindi_s.csv")

Creating CSV from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

8719